# PyCity School District Analysis

In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load
school_data_to_load = "Resources/schools_data_res.csv"
student_data_to_load = "Resources/students_data_res.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# District Summary

In [2]:
# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

In [3]:
# Calculate the number of Schools and the number of Students
total_schools = len(school_data)
total_students = len(student_data)

In [4]:
# Calculate the Total Budget
total_budget = '${:,.2f}'.format(sum(school_data["budget"]))

In [5]:
# Calculate the Average Math, Average Reading Score and the Overall Passing grade
avg_math_score = student_data["math_score"].mean()
avg_reading_score = student_data["reading_score"].mean()
avg_overall_score = (avg_math_score+avg_reading_score)/2

#Round to nearest integer for readabiliby
avg_math_score = round(avg_math_score)
avg_reading_score = round(avg_reading_score)
avg_overall_score = round(avg_overall_score)

In [6]:
# Calculate the percentage of students with a passing math score and a passing reading score (70 or greater)
students_math_pass = student_data.loc[student_data["math_score"] >= 70 ].count()
students_read_pass = student_data.loc[student_data["reading_score"] >= 70 ].count()

pct_math_pass = students_math_pass[0]/total_students*100
pct_read_pass = students_read_pass[0]/total_students*100
pct_overall_pass = (pct_math_pass+pct_read_pass)/2

#Round to 1 digit for readability
pct_math_pass = round(pct_math_pass,1)
pct_read_pass = round(pct_read_pass,1)
pct_overall_pass = round(pct_overall_pass,1)

In [7]:
#Print results to screen
district_results = pd.DataFrame({"Total Schools": [total_schools],
                                   "Total Students": [total_students],
                                   "Total Budget": [total_budget],
                                   "Average Math Score": [avg_math_score],
                                   "Average Reading Score": [avg_reading_score],
                                   "% Passing Math": [pct_math_pass],
                                   "% Passing Reading": [pct_read_pass],
                                   "% Passing Overall": [pct_overall_pass]
                                   })

district_results

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
0,15,39170,"$24,649,428.00",79,82,75.0,85.8,80.4


#  School Summary

In [8]:
#School Summary - Create dataframe based on student data grouped by School Name
grouped_by_school_df = student_data.groupby(['school_name'])

In [9]:
#Create a dataframe with 'school' being the index to which all additional values can be linked
school_summary = pd.DataFrame(student_data["school_name"].value_counts())

school_summary.reset_index(inplace=True)
school_summary.columns = ["School Name", "Total Students"]
school_summary = school_summary.set_index("School Name")

In [10]:
#Calculate number of students, their average reading & math score by group (=school)
#Then append to School Summary table
total_students = grouped_by_school_df["Student ID"].count()
school_summary["Students Total for math"] = total_students 

avg_reading_score = grouped_by_school_df["reading_score"].mean()
school_summary["Average Reading Score"] = avg_reading_score 

avg_math_score = grouped_by_school_df["math_score"].mean()
school_summary["Average Math Score"] = avg_math_score 

In [11]:
#Create a list for those student with Math scores => 70
total_students_math_over_70 = student_data.loc[student_data["math_score"] >= 70]
#Then group by school
grouped_by_school_for_math = total_students_math_over_70.groupby(['school_name'])
number_of_math_students_70 = grouped_by_school_for_math["Student ID"].count()
school_summary["Students that pass math"] = number_of_math_students_70
#Then find the percentage that passed compared to the overall student count per school
pct_passing_math_70 = number_of_math_students_70/total_students
school_summary["% Math Passed"] = round(pct_passing_math_70*100,2)

In [12]:
#Create a list for those student with Reading scores => 70
total_students_read_over_70 = student_data.loc[student_data["reading_score"] >= 70]
#Then group by school
grouped_by_school_for_reading = total_students_read_over_70.groupby(["school_name"])
number_of_reading_students_70 = grouped_by_school_for_reading["Student ID"].count()
school_summary["Students that pass reading"] = number_of_reading_students_70
#Then find the percentage that passed compared to the overall student count per school
pct_passing_reading_70 = number_of_reading_students_70/total_students
school_summary["% Reading Passed"] = round(pct_passing_reading_70*100,2)

In [13]:
#Lastly determine the Overall - or average - of the math and reading scores
pct_passing_overall = round((pct_passing_reading_70 + pct_passing_math_70)/2*100,2)
school_summary["% Overall Passed"] = pct_passing_overall

In [30]:
#Again on the school level, create a dataframe for collecting data from school_data file
about_the_school = pd.DataFrame(school_data[["school_name","type","budget"]])
about_the_school.columns = ["School Name", "Type","Budget"]
about_the_school = about_the_school.set_index("School Name")
about_the_school.dtypes

school_summary["Budget"] = about_the_school["Budget"].map('${:,.2f}'.format)
school_summary["Type"] = about_the_school["Type"]
school_summary["Budget per Student"] = (about_the_school["Budget"]/school_summary["Total Students"]).map('${:,.2f}'.format)

In [34]:
#Finanlize the School Summary by selecting columns in specific order
school_final_summary = school_summary[['Type','Total Students', 'Budget', 'Budget per Student', 'Average Reading Score','Average Math Score',
         '% Math Passed', '% Reading Passed', '% Overall Passed']].sort_values(by="School Name")

# Top Performing School (by passing rate)

In [33]:
#Select TOP 5 schools based on the % Overall Passed
school_final_summary.sort_values(by="% Overall Passed",ascending=False).head(5) 

,Type,Total Students,Budget,Budget per Student,Average Reading Score,Average Math Score,% Math Passed,% Reading Passed,% Overall Passed
School Name,,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.975780,83.061895,94.13,97.04,95.59
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.848930,83.418349,93.27,97.31,95.29
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.816757,83.351499,93.39,97.14,95.27
Pena High School,Charter,962,"$585,858.00",$609.00,84.044699,83.839917,94.59,95.95,95.27
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.989488,83.274201,93.87,96.54,95.20


# Bottom Performing Schools (by passing rate)

In [18]:
#Select the worst 5 schools based on the % Overall Passed
school_final_summary.sort_values(by="% Overall Passed",ascending=True).head(5)

,Type,Total Students,Budget,Budget per Student,Average Reading Score,Average Math Score,% Math Passed,% Reading Passed,% Overall Passed
School Name,,,,,,,,,
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,80.744686,76.842711,66.37,80.22,73.29
Figueroa High School,District,2949,"$1,884,411.00",$639.00,81.158020,76.711767,65.99,80.74,73.36
Huang High School,District,2917,"$1,910,635.00",$655.00,81.182722,76.629414,65.68,81.32,73.50
Johnson High School,District,4761,"$3,094,650.00",$650.00,80.966394,77.072464,66.06,81.22,73.64
Ford High School,District,2739,"$1,763,916.00",$644.00,80.746258,77.102592,68.31,79.30,73.80


# Math Scores by Grade

In [19]:
#Create a dataframe with 'school' being a summary level again
scores_by_grade_df = pd.DataFrame(student_data["school_name"].value_counts())

In [20]:
#Per grade level, summarize the math score and count the number of students to compute an average per grade
get_9TH_math = student_data.loc[student_data["grade"] == "9th", ["school_name", "math_score"]]
get_9TH_math_gp = get_9TH_math.groupby(['school_name'])
scores_by_grade_df["9th_grades"] = get_9TH_math_gp.sum()
scores_by_grade_df["9th_students"] = get_9TH_math_gp.count()
scores_by_grade_df["9th"] = scores_by_grade_df["9th_grades"]/scores_by_grade_df["9th_students"]

get_10TH_math = student_data.loc[student_data["grade"] == "10th", ["school_name","math_score"]]
get_10TH_math_gp = get_10TH_math.groupby(['school_name'])
scores_by_grade_df["10th_grades"] = get_10TH_math_gp.sum()
scores_by_grade_df["10th_students"] = get_10TH_math_gp.count()
scores_by_grade_df["10th"] = scores_by_grade_df["10th_grades"]/scores_by_grade_df["10th_students"]

get_11TH_math = student_data.loc[student_data["grade"] == "11th", ["school_name","math_score"]]
get_11TH_math_gp = get_11TH_math.groupby(['school_name'])
scores_by_grade_df["11th_grades"] = get_11TH_math_gp.sum()
scores_by_grade_df["11th_students"] = get_11TH_math_gp.count()
scores_by_grade_df["11th"] = scores_by_grade_df["11th_grades"]/scores_by_grade_df["11th_students"]

get_12TH_math = student_data.loc[student_data["grade"] == "12th", ["school_name","math_score"]]
get_12TH_math_gp = get_12TH_math.groupby(['school_name'])
scores_by_grade_df["12th_grades"] = get_12TH_math_gp.sum()
scores_by_grade_df["12th_students"] = get_12TH_math_gp.count()
scores_by_grade_df["12th"] = scores_by_grade_df["12th_grades"]/scores_by_grade_df["12th_students"]

scores_by_grade_final = scores_by_grade_df[['9th','10th','11th','12th']].sort_index(ascending=True)
scores_by_grade_final

,9th,10th,11th,12th
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248
Pena High School,83.625455,83.372000,84.328125,84.121547


# Reading Scores by Grade

In [21]:
#Per grade level, summarize the reading score and count the number of students to compute an average per grade
get_9TH_reading = student_data.loc[student_data["grade"] == "9th", ["school_name", "reading_score"]]
get_9TH_reading_gp = get_9TH_reading.groupby(['school_name'])
scores_by_grade_df["9th_grades"] = get_9TH_reading_gp.sum()
scores_by_grade_df["9th_students"] = get_9TH_reading_gp.count()
scores_by_grade_df["9th"] = scores_by_grade_df["9th_grades"]/scores_by_grade_df["9th_students"]

get_10TH_reading = student_data.loc[student_data["grade"] == "10th", ["school_name","reading_score"]]
get_10TH_reading_gp = get_10TH_reading.groupby(['school_name'])
scores_by_grade_df["10th_grades"] = get_10TH_reading_gp.sum()
scores_by_grade_df["10th_students"] = get_10TH_reading_gp.count()
scores_by_grade_df["10th"] = scores_by_grade_df["10th_grades"]/scores_by_grade_df["10th_students"]

get_11TH_reading = student_data.loc[student_data["grade"] == "11th", ["school_name","reading_score"]]
get_11TH_reading_gp = get_11TH_reading.groupby(['school_name'])
scores_by_grade_df["11th_grades"] = get_11TH_reading_gp.sum()
scores_by_grade_df["11th_students"] = get_11TH_reading_gp.count()
scores_by_grade_df["11th"] = scores_by_grade_df["11th_grades"]/scores_by_grade_df["11th_students"]

get_12TH_reading = student_data.loc[student_data["grade"] == "12th", ["school_name","reading_score"]]
get_12TH_reading_gp = get_12TH_reading.groupby(['school_name'])
scores_by_grade_df["12th_grades"] = get_12TH_reading_gp.sum()
scores_by_grade_df["12th_students"] = get_12TH_reading_gp.count()
scores_by_grade_df["12th"] = scores_by_grade_df["12th_grades"]/scores_by_grade_df["12th_students"]

scores_by_grade_final = scores_by_grade_df[['9th','10th','11th','12th']].sort_index(ascending=True)
scores_by_grade_final

,9th,10th,11th,12th
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564
Pena High School,83.807273,83.612000,84.335938,84.591160
